# Data Preprocessing Turkey Earthquake data (Omdena TR project)

In [1]:
import pandas as pd
import numpy as np

from datetime import datetime, timedelta

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
df = pd.read_csv(r"final.csv",sep=",", index_col=0) #, encoding='cp1252') 
df["Date_Time"] = pd.to_datetime(df.Date +" "+ df["Origin Time"])
df.head(2)

,Date,Origin Time,Latitude,Longitude,Depth(km),xM,MD,ML,Mw,Ms,Mb,Type,Location,City,is_out_of_Turkey,id,ID_,NAME1_,NAME2_,PARTS_,POINTS_,LENGTH_,AREA_,geometry,Direction,Date_Time
0,1900.09.20,00:00:01.00,37.8,29.1,5.0,5.0,5.0,0.0,NaN,0.0,0.0,Ke,DENIZLI (DENIZLI) [North East 2.3 km],DENIZLI,False,NaN,ASITUR003002002,Denizli,Denizli,1,164,364.9901,4621.875,POINT (29.1 37.8),North East 2.3 km,1900-09-20 00:00:01
1,1904.01.01,11:38:00.00,37.8,29.1,20.0,4.9,4.8,4.8,4.9,4.8,4.9,Ke,DENIZLI (DENIZLI) [North East 2.3 km],DENIZLI,False,NaN,ASITUR003002002,Denizli,Denizli,1,164,364.9901,4621.875,POINT (29.1 37.8),North East 2.3 km,1904-01-01 11:38:00


In [3]:
df_orig = df.copy()

#### Inspecting features

In [4]:
variables = pd.DataFrame(columns=['Variable',"Dtype", '# Unique Values','Values'])

for i, var in enumerate(df.columns):
    variables.loc[i] = [var, df[var].dtype, df[var].nunique(), 
                        df[var].value_counts(dropna=False).index.tolist()]    ## Values are sorted according to their frequency in the data
variables

,Variable,Dtype,# Unique Values,Values
0,Date,object,13081,"[2011.10.23, 2011.10.24, 2002.02.03, 2011.10.2..."
1,Origin Time,object,40210,"[00:00:01.00, 00:00:00.00, 11:15:05.00, 09:12:..."
2,Latitude,float64,12914,"[39.0, 37.0, 39.3, 39.1, 39.4, 37.1, 40.7, 39...."
3,Longitude,float64,18058,"[29.1, 29.5, 29.0, 29.3, 27.7, 29.4, 29.2, 27...."
4,Depth(km),float64,577,"[5.0, 10.0, 0.0, 8.0, 6.0, 7.0, 9.0, 1.0, 4.0,..."
5,xM,float64,46,"[3.0, 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, ..."
6,MD,float64,41,"[0.0, 3.0, 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, ..."
7,ML,float64,46,"[0.0, 3.0, 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, ..."
8,Mw,float64,46,"[nan, 3.2, 3.3, 3.1, 0.0, 3.4, 3.0, 3.5, 3.8, ..."
9,Ms,float64,43,"[0.0, 4.5, 4.6, 4.8, 4.7, 4.2, 4.0, 4.9, 5.0, ..."


* xM, MD, ML, Mw, Ms, Mb : contains values of different magnitude types. We will be keeping all of them at this stage. (For the other projects published on the web (Kaggle, Medium etc.) and data visualizations on koeri.boun.edu web page xM is used as the Magnitude variable. We also asked Gijs (on slack) to understand if Mw is the actual Magnitude variable. ) 
* Type column indicates whether the observation is an earthquake (Ke) or any suspected explosion (Sm). --- We will binary encode it as we discussed on the last meeting.
* Location will be kept for EDA process, and will be dropped during modeling. 
* is_out_of_Turkey was built by using the details in Location column, after using geopandas it became redundant and will be removed.
* id column has nan values for all records and will be removed.
* City, NAME1_ and NAME2_ contain city values. The difference between NAME1_ and NAME2_ is NAME2_contains Turkish characters (like ü,ğ ...). City is a feature obtained by getting city names from Location column and NAME1_, NAME2_ are obtained from geopandas. As we agreed on the meeting 20th of July, for the records which have inconsistencies in City and NAME1_ features we will use geopy library to get the City values. 
* As we agreed on the meeting 20th of July, the geopandas features ID_, LENGTH_, PARTS_, POINTS_ will be removed.
* AREA and geometry will be kept for modeling (as we agreed on the meeting 20th of July).
* "Direction" will be removed from the data set, as we have discussed in the meeting on 24th of July.

In [5]:
cols_to_drop = ["Date","Origin Time","is_out_of_Turkey","id","NAME1_","NAME2_",
                "ID_", "LENGTH_", "PARTS_", "POINTS_", "Direction"]

### Appending the records which are near to Turkey but filtered out by geopandas 
As we have agreed on the meetings on 20th and 24th of July...

For detailed explanation please check the "Omdena_Turkey_chapter__Preprocess.ipynb" file shared on slack (task-1-data channel on 20th of July).

Loading the previous dataset (containing all records both inside and outide of Turkey): 

In [6]:
df_pre = pd.read_csv(r"Turkey_earthquakes_koeri_1900_2022_rev1.csv",sep=",", index_col=0) #, encoding='cp1252') 
df_pre["Date_Time"] = pd.to_datetime(df_pre.Date +" "+ df_pre["Origin Time"])
df_pre.head(1)

,Event ID,Date,Origin Time,Latitude,Longitude,Depth(km),xM,MD,ML,Mw,Ms,Mb,Type,Location,City,is_out_of_Turkey,Date_Time
0,19000920000001,1900.09.20,00:00:01.00,37.8,29.1,5.0,5.0,5.0,0.0,NaN,0.0,0.0,Ke,DENIZLI (DENIZLI) [North East 2.3 km],DENIZLI,False,1900-09-20 00:00:01


Filtering the records of previous data set which are in Turkey or in Marmara Sea (according to the "Location" column of the dataset): 

In [7]:
## getting Date_Time values of the earthquakes as the unique idenditifers
## Earthquakes in Turkey according to the Location column
in_TR_pre = set(df_pre.loc[df_pre.is_out_of_Turkey==False, "Date_Time"])

### Earthquakes in Marmara Sea
in_Marmara = df_pre.loc[df_pre.Location.str.contains("MARMARA"), "Date_Time"]

in_TR_pre.update(in_Marmara)   ## all earthquakes in Turkey (according to the Location column)
len(in_TR_pre)

42823

Comparing its unique identifier (Date_Time) with the final.csv df:

In [8]:
# all unique records in final.csv
in_TR_Final = set(df["Date_Time"])

# earthquake ids that are in previous df but not in final.csv:
in_pre_but_not_in_final = in_TR_pre - in_TR_Final
len(in_pre_but_not_in_final)

2806

The below code block creates the data set of 2806 records. 

In [9]:
df_to_get_geopandas_feats = df_pre.loc[(df_pre.Date_Time.isin(in_pre_but_not_in_final))]
# df_to_get_geopandas_feats.to_csv("records_to_add_geopandas_feats")

##### Appending these 2806 records and appending to dataset
To append these records to our dataset we need to create the same features as df..

I used the code below to get the geopandas features, but since these earthquakes are outside of Turkey geopandas bring NAN values for all features (so there is no need to use below code). We will append 2806 records to our data and will be imputing them in "Handling missing values" section.

In [10]:
df_to_get_geopandas_feats.drop("Event ID", axis=1, inplace=True)

In [11]:
df = df.append(df_to_get_geopandas_feats, ignore_index=True)

### Deciding the feature to be used as Magnitude value
We will be keeping all of the magnitude features in our dataset (at this stage). I would like to keep the text below to share how others selected the target (magnitude) variable.

Accoridng to http://www.koeri.boun.edu.tr/sismo/2/earthquake-catalog/ wep page:
* xM : Biggest magnitude value in specified magnitude values (MD, ML, Mw, Ms and Mb).
* MD, ML, Mw, Ms or Mb : Magnitude types (MD: Duration, ML: Local, Mw: Moment, Ms: Surface wave, Mb: Body-wave). 0.0 (zero) means no calculation for that type of magnitude.

As I understand form the last statement on that web page (the last sentence of 3.b), using xM as the magnitude metric will be reasonable. Because the data provider used this feature on their visualiztion.

Also on the blog articles below, "xM" has been used as the Magnitude variable:
* [Turkey Earthquake Analysis (1915-2020) - Ata Saygin](https://www.kaggle.com/code/atasaygin/turkey-earthquake-analysis-1915-2020)
* [Explanatory Analysis of the Earthquakes in Turkey - Nilay Cicekli](https://github.com/nilaycicekli/earthquake-EDA-turkey/blob/master/Earthquake%20Analysis.ipynb)
* [Earthquake Parameter Prediction with Linear Regression - Fatma Elik](https://medium.com/analytics-vidhya/earthquake-parameter-prediction-with-linear-regression-c86e5abff79f) 

#### Binary encoding of Type feature

In [12]:
df.Type.value_counts(normalize=True)

Ke    0.997375
Sm    0.002625
Name: Type, dtype: float64

In [13]:
df.Type = df.Type.map({"Ke":1,"Sm":0})

### Building a reliable City Feature
City feature on the data set is obtianed from Location column and NAME1_ is obtianed from geopandas library. As seen below they have inconsistencies in 10057 records. As agreed on 20th of July we will use geopy library to get reliable City values for the records having that inconsistency problem.

In [14]:
df.loc[df.NAME1_.str.lower() != df.City.str.lower()].shape    #head(20) 

(10057, 26)

We will copy city values obtained from Location column to City_Location, and create a new City column having the proper City values: 

In [18]:
df["City_Location"] = df["City"].copy()
df["City"] = np.nan

In [19]:
# getting the indexes with City values having inconsistency problem or not
rows_city_location_is_okay = df.loc[df.NAME1_.str.lower() == df.City_Location.str.lower()].index
rows_use_geopy_for_city    = df.loc[df.NAME1_.str.lower() != df.City_Location.str.lower()].index

In [20]:
# For the records having same value in City_Location and NAME1_:
df.loc[rows_city_location_is_okay, "City"] = df["City_Location"]

#### Using Geopy 

Below I couldn't get city values for all records (43K) because it took about 1.5 hours for 10K records to get the location detials from the api (geoapiExercises). That's why I got only the ones having inconsitencies between City and NAME1_.

In [ ]:
from geopy.geocoders import Nominatim

In [ ]:
# initialize Nominatim API
geolocator = Nominatim(user_agent="geoapiExercises")

In [ ]:
df["geopy_location"] =np.nan

for index, row in df.loc[rows_use_geopy_for_city].iterrows():
    df.loc[index ,"geopy_location"] = str(geolocator.reverse(str(row.Latitude)+","+str(row.Longitude)))

In [23]:
(df.geopy_location.notnull()).sum()

10057

In [24]:
def get_city(geopy_output):
    geopy_output_value = str(geopy_output)
    end_of_region   = geopy_output_value.find("Bölgesi")
    start_of_region = geopy_output_value[:end_of_region].rfind(",") 
    
    start_of_city   = geopy_output_value[:start_of_region].rfind(",")+1
    geopy_output_value[start_of_city : start_of_region].strip() 
    city = geopy_output_value[start_of_city : start_of_region].strip()
    return city

In [25]:
df.loc[rows_use_geopy_for_city, "City"]= df["geopy_location"].apply(get_city)

In [26]:
df.loc[(df["geopy_location"].notnull()) 
       & (df["geopy_location"].str.find("Bölgesi")==-1),"City"].value_counts().head()

پارێزگای ئازەربایجانی ڕۆژاوا    152
ھەرێمی کوردستان                  59
685 00                           18
Շիրակի մարզ                      15
محافظة إدلب                      12
Name: City, dtype: int64

In [27]:
len(df.loc[(df["geopy_location"].notnull()) 
       & (df["geopy_location"].str.find("Bölgesi")==-1)])

394

There are 394 records of which city names obtained from geopy are not Turkish. Replacing them with the values in City_Locaiton column (obtained from the Location column) may be reasonable:

In [28]:
for index, row in df.loc[(df["geopy_location"].notnull()) 
                   & (df["geopy_location"].str.find("Bölgesi")==-1)].iterrows():
    
    if row["City_Location"] != "out_of_Turkey":     
        df.loc[index ,"City"] = row["City_Location"]  
    elif row["NAME1_"]!=np.nan:
        df.loc[index ,"City"] = row["NAME1_"].upper()  

In [29]:
df["City"] = df["City"].str.upper()

In [30]:
df.City = (df.City.str.replace("i","I").str.replace("İ","I")
             .str.replace("Ö","O")
             .str.replace("Ü","U")
             .str.replace("Ç","C")
             .str.replace("Ğ","G")
             .str.replace("Â","A")
             .str.replace("Ş","S"))

In [31]:
sorted(list(df.City.unique()))

['ADANA',
 'ADIYAMAN',
 'AFYONKARAHISAR',
 'AGRI',
 'AKSARAY',
 'AMASYA',
 'ANKARA',
 'ANTALYA',
 'ARDAHAN',
 'ARTVIN',
 'AYDIN',
 'BALIKESIR',
 'BARTIN',
 'BATMAN',
 'BAYBURT',
 'BILECIK',
 'BINGOL',
 'BITLIS',
 'BOLU',
 'BURDUR',
 'BURSA',
 'CANAKKALE',
 'CANKIRI',
 'CORUM',
 'DENIZLI',
 'DIYARBAKIR',
 'DUZCE',
 'EDIRNE',
 'ELAZIG',
 'ERZINCAN',
 'ERZURUM',
 'ESKISEHIR',
 'GAZIANTEP',
 'GIRESUN',
 'GUMUSHANE',
 'HAKKARI',
 'HATAY',
 'IGDIR',
 'ISPARTA',
 'ISTANBUL',
 'IZMIR',
 'IZMIR-USAK YOLU',
 'KAHRAMANMARAS',
 'KALE',
 'KAPISUYU',
 'KARABUK',
 'KARAMAN',
 'KARS',
 'KASTAMONU',
 'KAYSERI',
 'KILIS',
 'KIRIKKALE',
 'KIRKLARELI',
 'KIRSEHIR',
 'KOCAELI',
 'KONYA',
 'KUTAHYA',
 'MALATYA',
 'MANISA',
 'MARDIN',
 'MERSIN',
 'MEYDAN',
 'MUGLA',
 'MUS',
 'NEVSEHIR',
 'NIGDE',
 'ORDU',
 'OSMANIYE',
 'RIZE',
 'SAKARYA',
 'SAMSUN',
 'SANLIURFA',
 'SIIRT',
 'SINOP',
 'SIRNAK',
 'SIVAS',
 'TEKIRDAG',
 'TOKAT',
 'TRABZON',
 'TUNCELI',
 'TURKIY',
 'USAK',
 'VAN',
 'YALOVA',
 'YAYIKDAMLAR',
 'YO

According to the output above some recoreds have city names as 'IZMIR-USAK YOLU','KALE','KAPISUYU','KAPISUYU','TURKIY','YAYIKDAMLAR','MEYDAN'. They are actually not cities in Turkey. They need to be replaced:

In [32]:
wrong_city_names = ['IZMIR-USAK YOLU','KALE','KAPISUYU','MEYDAN','TURKIY','YAYIKDAMLAR']

In [33]:
df.loc[df.City.isin(wrong_city_names), "City_Location"].unique()

array(['IZMIR', 'ANTALYA', 'HATAY'], dtype=object)

So their City values will be replaced with City_Location:

In [34]:
df.loc[df.City.isin(wrong_city_names), "City"] = df["City_Location"]

##### Replacing AREA_ values according to new City names
AREA_ is the feature generated by geopandas and it shows the AREA_ values of each cities (as we understand from the data). Above we have changed city values of 10K records and we need to change their AREA_ values, too (assign the AREA_ values of their new City values):

Before starting we will create a backup feature for AREA_ as AREA_backup and will drop it later:

In [35]:
df["AREA_backup"] = df["AREA_"].copy()

Getting the correct city and area matches:

In [36]:
city_area_matches = df.loc[rows_city_location_is_okay, ["City","AREA_"]]  \
                                .drop_duplicates().set_index("City").to_dict()

In [37]:
df.loc[rows_use_geopy_for_city,"AREA_"] = np.nan

In [38]:
for index,row in df.loc[rows_use_geopy_for_city].iterrows():
    try:
        area_of_city = city_area_matches['AREA_'][row["City"]]    
        df.loc[index, "AREA_"] = area_of_city
    except:
        continue

Verifying if any record had problem with imputation:

In [39]:
df.loc[df.AREA_.isnull(), "City"].unique()

array(['AFYONKARAHISAR'], dtype=object)

In [40]:
set(df.City) - set(df.loc[rows_city_location_is_okay,"City"]) 

{'AFYONKARAHISAR'}

AFYONKARAHISAR (or AFYON) is not in the records of rows_city_location_is_okay, we need to get it manually. It is stated as Afyon in NAME1_ column: 

In [41]:
df.loc[df.NAME1_=="Afyon", "AREA_backup"].unique()

array([5760.035])

In [42]:
df.loc[df.City=="AFYONKARAHISAR", "AREA_"] = 5760.035

The features built during the process of getting the city names will de dropped:

In [43]:
cols_to_drop.extend(["City_Location","geopy_location","AREA_backup"])

#### Extracting Date and Time Features

In [44]:
df["Year"]  = df.Date_Time.dt.year
df["Month"] = df.Date_Time.dt.month
df["Hour"]  = df.Date_Time.dt.hour

### Dropping the unnecessary features

In [45]:
cols_to_drop

['Date',
 'Origin Time',
 'is_out_of_Turkey',
 'id',
 'NAME1_',
 'NAME2_',
 'ID_',
 'LENGTH_',
 'PARTS_',
 'POINTS_',
 'Direction',
 'City_Location',
 'geopy_location',
 'AREA_backup']

In [46]:
df.drop(cols_to_drop,axis=1, inplace=True)
df.head(1)

,Latitude,Longitude,Depth(km),xM,MD,ML,Mw,Ms,Mb,Type,Location,City,AREA_,geometry,Date_Time,Year,Month,Hour
0,37.8,29.1,5.0,5.0,5.0,0.0,NaN,0.0,0.0,1,DENIZLI (DENIZLI) [North East 2.3 km],DENIZLI,4621.875,POINT (29.1 37.8),1900-09-20 00:00:01,1900,9,0


### Handling missing values

In [47]:
df.isnull().sum()

Latitude         0
Longitude        0
Depth(km)        0
xM               0
MD               0
ML               0
Mw           37659
Ms               0
Mb               0
Type             0
Location         0
City             0
AREA_            0
geometry      2806
Date_Time        0
Year             0
Month            0
Hour             0
dtype: int64

'geometry' is the geopandas features, they are null for the 2806 records we appended above. We can impute them by cretaing the same structure we observed on the non-null records.  (AREA_ is the other geopandas feature, for the 2806 records we don't see NaN here because updated them above..)

Creating the "geometry" feature values:

In [48]:
for index,row in df.loc[df.geometry.isnull()].iterrows():
    ## Building exactly the same structure as "geometry" column of geopandas
    long = int(row["Longitude"]) if (row["Longitude"]).is_integer() else row["Longitude"]
    lat  = int(row["Latitude"]) if (row["Latitude"]).is_integer() else row["Latitude"]
    
    df.loc[index, "geometry"] = "POINT ("+str(long)+" "+str(lat)+")"

In [49]:
df.Mw.isnull().sum() / df.shape[0]

0.8670795726653159

86% of records have null vaue in MW. I left them as is for now, because couldn't figure out how to impyte in a proper way. 

### Getting last version of the dataset -- csv